In [ ]:
import iris
iris.FUTURE.netcdf_promote = True


url = "http://www.smast.umassd.edu:8080/thredds/dodsC/FVCOM/NECOFS/Forecasts/NECOFS_GOM3_FORECAST.nc"
cubes = iris.load_raw(url)

print(cubes)

In [ ]:
cube = cubes.extract_strict("sea_water_potential_temperature")

In [ ]:
import pyugrid
import matplotlib.tri as tri


def get_mesh(cube, url):
    ug = pyugrid.UGrid.from_ncfile(url)
    cube.mesh = ug
    cube.mesh_dimension = 1
    return cube
    
def get_triang(cube):
    lon = cube.mesh.nodes[:, 0]
    lat = cube.mesh.nodes[:, 1]
    nv = cube.mesh.faces
    return tri.Triangulation(lon, lat, triangles=nv)

In [ ]:
%matplotlib inline

import numpy as np
import numpy.ma as ma
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

def plot_model(cube, time_idx=-1, depth_idx=None):
    lon = cube.mesh.nodes[:, 0]
    lat = cube.mesh.nodes[:, 1]
    triang = get_triang(cube)
    data = cube[time_idx, ...].data
    if depth_idx is not None:
        data = data[depth_idx, ...]
    print(data.shape)
    data = ma.masked_invalid(data)

    fig, ax = plt.subplots(figsize=(7, 7),
                           subplot_kw=dict(projection=ccrs.PlateCarree()))
    ax.set_extent([lon.min(), lon.max(), lat.min(), lat.max()])
    ax.coastlines()
    levs = np.arange(-1, 5, 0.2)
    cs = ax.tricontourf(triang, data, levels=levs)
    fig.colorbar(cs)
    ax.tricontour(triang, data, colors='k',levels=levs)
    gl = ax.gridlines(draw_labels=True)
    gl.xlabels_top = gl.ylabels_right = False
    return fig, ax

In [ ]:
cube = get_mesh(cube, url)

fig, ax = plot_model(cube, time_idx=-1, depth_idx=0)

In [ ]:
from iris.analysis import trajectory

ax.plot([-69, -63], [42, 44])

waypoints = [{'longitude': -69, 'latitude': 42}, {'longitude': -36, 'latitude': 44}]

traj = trajectory.Trajectory(waypoints)

lons = [d['longitude'] for d in traj.sampled_points]
lats = [d['latitude'] for d in traj.sampled_points]

sampled_points = [('longitude', lon),
                  ('latitude', lat)]

In [ ]:
c = cube[-1, ...]

for aux in c.aux_factories:
    c.remove_aux_factory(aux)

In [ ]:
glider = iris.analysis.trajectory.interpolate(c, sampled_points, method="nearest")